# Write a python program to Scrape the below-mentioned site and bring in the list of the first 5 postings under the "Search Postings" heading containing the following fields: Est. Value Notes, Description, Closing Date 

## Import essential modules/libraries

In [1]:
%pip install selenium
%pip install webdriver-manager


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


## definig xpaths and fields for csv

In [3]:
# field names
fields = ['Est. Value Notes', 'Description', 'Closing Date']

In [4]:
rows_xpaths = ['//*[@id="table_id"]/tbody/tr[1]/td[4]/div/a',
          '//*[@id="1"]/td[4]/div/a',
          '//*[@id="2"]/td[4]/div/a',
          '//*[@id="3"]/td[4]/div/a',
          '//*[@id="4"]/td[4]/div/a' ] # first 5 rows

In [5]:
details_xpath = [
    # Est. Value Notes
    '//*[@id="current_project"]/div/div[2]/div/table/tbody/tr[3]/td[2]',

    # Description 
    '//*[@id="current_project"]/div/div[3]/div/table/tbody/tr[3]/td[2]',
    
    # Closing Date
    '//*[@id="current_project"]/div/div[2]/div/table/tbody/tr[1]/td[2]'
]

## Function to launch browser and get a given link

In [6]:

def launch_chrome(link, headless=False):
    ops = Options()  # create object of Options

    if headless:
        ops.add_argument('--headless')  # headless browser testing

    # prevents browser from closing when function is returned
    ops.add_experimental_option("detach", True)

    driver = webdriver.Chrome(options=ops,  service=Service(
        ChromeDriverManager().install()))

    driver.get(link)
    return driver


## Function that extracts element using xpath
- In try block it waits untill page is fully loaded then findes the slement with given xpath
- If timeout then none is returned mentioning that element was not found, either due to incorrect xpath or unabe to load page

In [7]:
def get_element(driver, path, timeout):
    element_present = None

    try:
        element_present = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.XPATH, path)))
        
        # element_present = EC.presence_of_element_located((By.XPATH, path))
        # print(element_present)
        # WebDriverWait(driver, timeout).until(element_present)

    except TimeoutException:
        print("Timed out waiting for page to load")
        print(path)
    finally:
        return element_present


# Get details function finds the required details:
- given an element it clicks the link in html tablw row
- extracts required details using xpath
- reloads page to go back to table 
- gets details for 5 rows iteratively as required.

In [8]:
def get_details(driver, element):
    element.click()

    est_value_notes = get_element(driver, details_xpath[0], timeout=5).text
    desc = get_element(driver, details_xpath[1], timeout=5).text
    closing = get_element(driver, details_xpath[2], timeout=5).text

    driver.refresh()

    return ([est_value_notes, desc, closing])

# Working block where in the above functions are executed in logical sequence to get the information and store in a csv file.  

In [9]:
tdriver = launch_chrome(
    "https://qcpi.questcdn.com/cdn/posting/?group=1950787&provider=1950787")


# data rows of csv file
rows = []

for i in range(5):
    # timeout in seconds
    element = get_element(tdriver, rows_xpaths[i], timeout=10)
    print(element)
    details = get_details(tdriver, element)
    print(details)
    rows.append(details)


with open('data.csv', 'a', encoding='utf8', newline='') as csv_file:

    # using csv.writer method from CSV package
    write = csv.writer(csv_file)
    write.writerow(fields)
    write.writerows(rows)

tdriver.close()
tdriver.quit()


<selenium.webdriver.remote.webelement.WebElement (session="8aefe3d973d671e67abc8523a11eb255", element="C07A409D67BF58C934D57F81D3CC95BF_element_111")>
['$1,000,000.00 to $2,500,000.00', 'reconstructing 3000 E at Foothill Road and adding a deceleration lane at E 2800 N', 'Tue, 06/06/2023 02:00 PM MDT']
<selenium.webdriver.remote.webelement.WebElement (session="8aefe3d973d671e67abc8523a11eb255", element="E23D84133F93D7AB549501C43CEFD54B_element_190")>
['$500,000.00 to $1,000,000.00', 'constructing a signal on SH-75, MP 121', 'Tue, 06/06/2023 02:00 PM MDT']
<selenium.webdriver.remote.webelement.WebElement (session="8aefe3d973d671e67abc8523a11eb255", element="0775A0326715C1BAAE75C63A9CE6FAF7_element_269")>
['$500,000 to $1,000,000', 'Reapplying water borne pavement markings to various highway routes', 'Tue, 06/06/2023 02:00 PM MDT']
<selenium.webdriver.remote.webelement.WebElement (session="8aefe3d973d671e67abc8523a11eb255", element="831C8F712EBD956876418DC901B1C6B6_element_342")>
['$500,0